<a href="https://colab.research.google.com/github/YASSIRAMRAOUI/Text-Mining/blob/main/Atelier_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Traduction assistée en NLTK***

# **Étape 1 : Préparation de l’environnement**

In [ ]:
import nltk

# Téléchargements nécessaires
nltk.download(['punkt', 'averaged_perceptron_tagger', 'wordnet', 'omw-1.4', 'stopwords', 'punkt_tab'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **Étape 2 : Saisie utilisateur**

In [ ]:
text = input("Veuillez entrer un paragraphe en anglais (minimum 3 phrases) :\n> ")

Veuillez entrer un paragraphe en anglais (minimum 3 phrases) :
> I took a short break after finishing the report.  The company plans to increase its profits.  She is reading an interesting book about history.


# **Étape 3 : Segmentation en phrases**

In [ ]:
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text) # On découpe le texte en phrases.
print("Phrases détectées :")
for s in sentences:
    print("-", s)

Phrases détectées :
- I took a short break after finishing the report.
- The company plans to increase its profits.
- She is reading an interesting book about history.


# **Étape 4 : Tokenisation + POS tagging**

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag

all_pos = []
for sent in sentences:
    tokens = word_tokenize(sent) # On découpe chaque phrase en mots (tokens).
    tagged = pos_tag(tokens) # On applique le POS tagging (catégorie grammaticale).
    all_pos.append(tagged)
    print("\nPhrase :", sent)
    print(tagged)


Phrase : I took a short break after finishing the report.
[('I', 'PRP'), ('took', 'VBD'), ('a', 'DT'), ('short', 'JJ'), ('break', 'NN'), ('after', 'IN'), ('finishing', 'VBG'), ('the', 'DT'), ('report', 'NN'), ('.', '.')]

Phrase : The company plans to increase its profits.
[('The', 'DT'), ('company', 'NN'), ('plans', 'VBZ'), ('to', 'TO'), ('increase', 'VB'), ('its', 'PRP$'), ('profits', 'NNS'), ('.', '.')]

Phrase : She is reading an interesting book about history.
[('She', 'PRP'), ('is', 'VBZ'), ('reading', 'VBG'), ('an', 'DT'), ('interesting', 'JJ'), ('book', 'NN'), ('about', 'IN'), ('history', 'NN'), ('.', '.')]


# **Étape 5 : Filtrage des mots significatifs**

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif treebank_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    else:
        return None

keywords_per_sentence = []
for tagged_sent in all_pos:
    keywords = [(w, get_wordnet_pos(p)) for w, p in tagged_sent if get_wordnet_pos(p)]
    keywords_per_sentence.append(keywords) # On garde seulement Noms, Verbes, Adjectifs.

print("Mots significatifs par phrase :")
for sent, kws in zip(sentences, keywords_per_sentence):
    print("\n-", sent)
    print(kws)

Mots significatifs par phrase :

- I took a short break after finishing the report.
[('took', 'v'), ('short', 'a'), ('break', 'n'), ('finishing', 'v'), ('report', 'n')]

- The company plans to increase its profits.
[('company', 'n'), ('plans', 'v'), ('increase', 'v'), ('profits', 'n')]

- She is reading an interesting book about history.
[('is', 'v'), ('reading', 'v'), ('interesting', 'a'), ('book', 'n'), ('history', 'n')]


# **Étape 6 : Désambiguïsation avec Lesk**

In [ ]:
from nltk.wsd import lesk

results = [] # On applique l’algorithme Lesk de NLTK pour trouver le bon sens selon le contexte.
for sent, kws in zip(sentences, keywords_per_sentence):
    tokens = word_tokenize(sent)
    word_info = []
    for word, wn_pos in kws:
        synset = lesk(tokens, word, wn_pos)
        if synset:
            definition = synset.definition()
            translations = [lemma.name() for lemma in synset.lemmas(lang='fra')]
            if not translations:
                translations = ["(pas de traduction disponible)"]
            word_info.append({
                "word": word,
                "pos": wn_pos,
                "definition": definition,
                "translations": translations
            })
    results.append({"sentence": sent, "keywords": word_info})

# **Étape 7 : Affichage final**

In [ ]:
print("\n=== Assistant de traduction ===\n")
for res in results:
    print("Phrase :", res["sentence"])
    for kw in res["keywords"]:
        print(f" - Mot : {kw['word']}")
        print(f"   > Sens (EN) : {kw['definition']}") # On propose à l’utilisateur le sens
        print(f"   > Traduction(s) FR : {', '.join(kw['translations'])}") # Traduction possible en français
    print("-" * 60)


=== Assistant de traduction ===

Phrase : I took a short break after finishing the report.
 - Mot : took
   > Sens (EN) : develop a habit
   > Traduction(s) FR : (pas de traduction disponible)
 - Mot : short
   > Sens (EN) : of speech sounds or syllables of relatively short duration
   > Traduction(s) FR : court
 - Mot : break
   > Sens (EN) : (geology) a crack in the earth's crust resulting from the displacement of one side with respect to the other
   > Traduction(s) FR : casser, cassure, fracture
 - Mot : finishing
   > Sens (EN) : cause to finish a relationship with somebody
   > Traduction(s) FR : achever, finir
 - Mot : report
   > Sens (EN) : a short account of the news
   > Traduction(s) FR : compte, description, histoire, rapport, renseignement, récit, étage
------------------------------------------------------------
Phrase : The company plans to increase its profits.
 - Mot : company
   > Sens (EN) : an institution created to conduct business
   > Traduction(s) FR : compagn

# ***Calcul des similarité entre documents texte***

# **Étape 1 : Installation & imports**

In [ ]:
!pip install googletrans==4.0.0-rc1 jaro-winkler

import nltk, jaro
from googletrans import Translator
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn

# Téléchargement des ressources nécessaires
nltk.download(['punkt', 'wordnet', 'omw-1.4'])

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 676.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=f15f3677ff1ee51877009a566327c79995656e996e0145f4fb3f7c42f6401311
  Stored in directory: /root/.cache/pip/wheels/95/0f/04/b17a72024b56a60e499ce1a6313d283ed5ba332407155bee03
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfr

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# **Étape 2 : Saisie utilisateur & traductions**

In [ ]:
text_en = input("Entrez un paragraphe en anglais (≥ 3 phrases) :\n> ")

# traduction manuelle par l'utilisateur
manual_fr = input("\nTraduction manuelle en français :\n> ")

# traduction automatique
translator = Translator()
auto_fr = translator.translate(text_en, src='en', dest='fr').text

print("\n=== Textes obtenus ===")
print("Texte original (EN) :", text_en)
print("Traduction manuelle :", manual_fr)
print("Traduction auto (Google) :", auto_fr)

Entrez un paragraphe en anglais (≥ 3 phrases) :
> Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.

Traduction manuelle en français :
> Le Lorem Ipsum est simplement du faux texte employé dans la composition et la mise en page avant impression. Il est le faux texte standard de l'industrie depuis le XVIe siècle, lorsqu'un imprimeur anonyme assembla une galée de caractères pour en faire un recueil de spécimens typographiques.

=== Textes obtenus ===
Texte original (EN) : Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.
Traduction manuelle : Le Lorem Ipsum est simplement du faux texte employé dans la c

# **Étape 3 : Similarité syntaxique**

**2.1 Jaro-Winkler (texte entier)**

In [ ]:
score_jaro = jaro.jaro_winkler_metric(manual_fr, auto_fr)
print(f"\n[Jaro-Winkler] Similarité globale : {score_jaro:.3f}")


[Jaro-Winkler] Similarité globale : 0.804


**2.2 Similarité par phrases (Jaccard + Jaro)**

On découpe les deux textes en phrases, puis on compare phrase par phrase avec Jaro-Winkler.
Ensuite, on applique le coefficient de Jaccard.

In [ ]:
def jaccard(text1, text2, theta=0.85):
    sents1 = sent_tokenize(text1)
    sents2 = sent_tokenize(text2)

    matches = 0
    total = len(sents1) + len(sents2)

    for s1 in sents1:
        for s2 in sents2:
            if jaro.jaro_winkler_metric(s1, s2) > theta:
                matches += 1
                break  # éviter double comptage

    return matches / (total - matches)

jac_score = jaccard(manual_fr, auto_fr)
print(f"[Jaccard sur phrases] Similarité : {jac_score:.3f}")

[Jaccard sur phrases] Similarité : 0.000


# **Étape 4 : Similarité sémantique**

On compare mot par mot en cherchant leur sens avec Lesk, puis on calcule la similarité avec Wu-Palmer (wup_similarity).

In [ ]:
def get_synsets(text):
    tokens = word_tokenize(text)
    synsets = []
    for token in tokens:
        syn = lesk(tokens, token)
        if syn:
            synsets.append(syn)
    return synsets

def semantic_distance(text1, text2):
    syn1 = get_synsets(text1)
    syn2 = get_synsets(text2)

    scores = []
    for s1 in syn1:
        best = 0
        for s2 in syn2:
            sim = s1.wup_similarity(s2)
            if sim and sim > best:
                best = sim
        if best > 0:
            scores.append(best)
    return sum(scores) / len(scores) if scores else 0

sem_score = semantic_distance(manual_fr, auto_fr)
print(f"\n[Wu-Palmer] Similarité sémantique : {sem_score:.3f}")


[Wu-Palmer] Similarité sémantique : 0.800


# ***Résumé***

In [ ]:
print("\n====== Résumé de la comparaison ======\n")
print("Texte original (EN) :", text_en)
print("Traduction manuelle :", manual_fr)
print("Traduction auto (Google) :", auto_fr)

print("\n--- Scores de similarité ---")
print(f"Similarité syntaxique (Jaro-Winkler global) : {score_jaro:.3f}")
print(f"Similarité syntaxique (Jaccard sur phrases) : {jac_score:.3f}")
print(f"Similarité sémantique (Wu-Palmer via Lesk) : {sem_score:.3f}")

print("\n--- Analyse rapide ---")
# Analyse Jaro
if score_jaro > 0.85:
    print("✅ Les deux traductions sont très proches au niveau syntaxique (Jaro-Winkler).")
else:
    print("⚠️ Différences notables dans la structure des phrases (Jaro-Winkler).")

# Analyse Jaccard
if jac_score > 0.75:
    print("✅ Les traductions contiennent majoritairement les mêmes phrases (Jaccard).")
else:
    print("⚠️ Plusieurs phrases diffèrent entre les deux traductions (Jaccard).")

# Analyse sémantique
if sem_score > 0.75:
    print("✅ Le sens général est bien conservé (forte similarité sémantique).")
else:
    print("⚠️ Possible divergence de sens entre les deux traductions (sémantique).")


====== Résumé de la comparaison ======

Texte original (EN) : Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.
Traduction manuelle : Le Lorem Ipsum est simplement du faux texte employé dans la composition et la mise en page avant impression. Il est le faux texte standard de l'industrie depuis le XVIe siècle, lorsqu'un imprimeur anonyme assembla une galée de caractères pour en faire un recueil de spécimens typographiques.
Traduction auto (Google) : Lorem Ipsum est simplement un texte muet de l'industrie de l'impression et de la composition.Lorem Ipsum est le texte manqué standard de l'industrie depuis les années 1500, lorsqu'une imprimante inconnue a pris une cuisine de type et l'a brouillée pour faire un livre de spécimen de type.

--- Scores de similarité ---
Similarité syntaxique (Jaro-Win